In [64]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Using for cleaning and Pre-Processing
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import pos_tag
import spacy
en = spacy.load('en_core_web_sm')
# To generate embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS,Phraser
from gensim.models import Word2Vec, KeyedVectors #To load the model
from cleantext import clean
#Visualisations
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Calibri"
import matplotlib.gridspec as gridspec
import os
from datetime import timedelta
from datetime import datetime
from time import time
from tqdm import tqdm
from collections import Counter
from concurrent.futures import ProcessPoolExecutor
from preprocesss import preprocess_batch
#To check for performance
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from keybert import KeyBERT
import torch
from pygtrie import CharTrie
from collections import Counter
stopword=list(stopwords.words('english'))

In [65]:
import torch
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained('Dhanush66/AntismetisimLargedata-finetuned-MLM')
model = BertModel.from_pretrained('Dhanush66/AntismetisimLargedata-finetuned-MLM')

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
#% matplotlib inline
pa='/Users/dhanushkikkisetti/Documents/Research Assistant/right_left_finetunemodel'
# Load pre-trained model tokenizer (vocabulary)
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model=BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at Dhanush66/AntismetisimLargedata-finetuned-MLM were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at Dhanush66/AntismetisimLargedata-finetuned-MLM and are newly initialized: ['bert

In [66]:
data=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Raza_data/Unmasking Antisemitism SRI Data Set - Reporting Layer.csv")
data=data[['Term or Phrase','Post Text']]
data['Term or Phrase'].unique()

array(['Cabal', 'Cosmopolitan Elite', 'Cultural Marxism', 'Deicide',
       'The Goyim Know', 'Holocough', 'Jewish Capitalist',
       'Jewish Communist', 'Jew Down', 'Jewish Lobby', 'New World Order',
       'Not the Real Jews', 'Rothschild', 'Soros', 'Zionist/Zio',
       'Zionist Occupied Government', nan], dtype=object)

In [67]:
data=data.drop_duplicates()

In [68]:
glossary=['cabal','cosmopolitan elite','cultural marxism','deicide','holocough','jewish capitalist','the goyim know',
           'jewish communist','jewish lobby','new world order','rothschild', 'soros','zionist',
         'zionist occupied government','jew down','not the real jews'] 
emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                                "]+", flags=re.UNICODE)

In [69]:
data=data[-data["Post Text"].isnull()]

In [70]:
lematizer=WordNetLemmatizer()
def lematize(text):
    text=text.split()
    lema=[]
    for i in text:
        lema.append(lematizer.lemmatize(i))
    return (" ".join(lema))

In [71]:
data['clean']=data['Post Text'].apply(lambda x:x.lower())
#removing the links:
data['clean']=data['clean'].apply(lambda x:re.sub(r"http\S+","",str(x)))
data['clean']=data['clean'].apply(lambda x:x.translate(str.maketrans("","",string.punctuation)))
#Getting the lematised text to get the original form of the word
data['lematize']=data['clean'].apply(lambda x:lematize(x))
#data['clean']=data['clean'].apply(lambda x:x.translate(str.maketrans("","",string.punctuation)))
uncleaned_texts=list(data["clean"])
uncleaned_lematised_text=list(data['lematize'])

In [72]:
terms=['access biden','access computers','deep state','dollar sent','every dollar','nancy pelosi','voting machines','world economic forum',
      'arrested made','bless tate','enemies humanity','every member','extinct deep','god bless','humanity satanic','instead every','member obiden',
      'needs extinct','new world order','poor souls','socialist party','souls murdered','started zionist','tate arrested','rootless cosmopolitan',
'frankfurt school','early part','found early','marxism found',
'origins political',
'commit decide',
'goyim know',
'know shut',
'shut goyim',
'know goyim',
'know vey',
'vey goyim',
'know kek',
'shlomo goyim',
'crumble shit',
'done goyim',
'kek shut',
'250k let',
'deal already',
'done deal',
'goyim shlomo',
'capitalist intellectuals',
'capitalist system',
'communist takeover',
'communist coup',
'coup attempt',
'chinese without',
'communist criminals',
'communist party',
'communist pedophiles',
'communist propaganda',
'communist rule',
'failed communist',
'kpd communist',
'reality ashkenazi',
'takeover communist',
'without putting',
'2004 directed',
'back 2004',
'big oil',
'burning copy',
'cartels send',
'control united',
'copy holy',
'crossed swords',
'gaza terror',
'holy quran',
'israel sanction',
'send israel',
'terror groups',
'united states',
'disposable small',
'small cog',
'also known',
'back better',
'forum declares',
'ild back',
'job read',
'machine job',
'order agenda',
'order lay',
'order machine',
'black tribes',
'mike tyson',
'created israel',
'focused like',
'gate redpill',
'khazarian cabal',
'khazarian mafia',
'klaus schwab',
'mafia minions',
'order agenda',
'owe money',
'owns runs',
'pay therefore',
'redpill world',
'runs created',
'sorry owe',
'state cabal',
'trillionaire jacob',
'zionist occupied government',
'occupied government',
'space time',
'time portal',
'time information',
'blood type',
'interfere behalf',
'time travel',
'advanced technology',
'caves space',
'children including',
'etc names',
'occupation government',
'spare one',
'travel agents',
'agent names',
'cave accesses',
'caves report',
'government judeocra',
'latin brown',
'media organizations']

In [73]:
len(terms)

126

In [74]:
new_terms=[]

for t in terms:
    for text in uncleaned_texts:
        if t in text:
            new_terms.append(t)
            break


In [75]:
new_terms

['deep state',
 'nancy pelosi',
 'voting machines',
 'world economic forum',
 'bless tate',
 'every member',
 'god bless',
 'new world order',
 'poor souls',
 'socialist party',
 'tate arrested',
 'rootless cosmopolitan',
 'frankfurt school',
 'early part',
 'goyim know',
 'know shut',
 'deal already',
 'done deal',
 'capitalist intellectuals',
 'capitalist system',
 'communist takeover',
 'communist coup',
 'coup attempt',
 'chinese without',
 'communist criminals',
 'communist party',
 'communist pedophiles',
 'communist propaganda',
 'communist rule',
 'failed communist',
 'kpd communist',
 'without putting',
 'big oil',
 'cartels send',
 'crossed swords',
 'gaza terror',
 'holy quran',
 'israel sanction',
 'terror groups',
 'united states',
 'small cog',
 'also known',
 'back better',
 'order machine',
 'black tribes',
 'mike tyson',
 'created israel',
 'focused like',
 'khazarian cabal',
 'khazarian mafia',
 'klaus schwab',
 'mafia minions',
 'owns runs',
 'pay therefore',
 'state

In [76]:
new_glossary=[]
for term in glossary:
    if len(term.split(" "))==1:
        new_glossary.append(term)
    elif len(term.split(" "))==2:
        new_glossary.append(term.split()[0])
        new_glossary.append(term.split()[1])
    else:
        terms=list(zip(term.split()[:-1],term.split()[1:]))
        pairs=[' '.join(i)  for i in terms]
        pairs.append(' '.join([term.split()[0],term.split()[-1]]))
        new_glossary+=pairs

In [77]:
for g in new_glossary:
    for t in new_terms:
        if g in t:
            new_terms.remove(t)

In [78]:
new_terms.remove("early part")

In [79]:
def extract_context_words_bigram(sentence, target_bigram, num_words_before, num_words_after):
    words = sentence.split()
    context_words = []
    if len(target_bigram.split())==4:
        for i in range(len(words)-3):
            fourgram=" ".join([words[i],words[i+1],words[i+2],words[i+3]])
            if fourgram==target_bigram:
                start_index = max(0, i - num_words_before)
                end_index = min(len(words), i + num_words_after + 4)
                context_words.append(words[start_index:end_index])            
    elif len(target_bigram.split())==3:
        for i in range(len(words) - 2):  # Loop through pairs of consecutive words
            trigram=" ".join([words[i],words[i+1],words[i+2]])
            if trigram == target_bigram: 
                start_index = max(0, i - num_words_before)
                end_index = min(len(words), i + num_words_after + 3)
                context_words.append(words[start_index:end_index])
    else:
        for i in range(len(words) - 1):  # Loop through pairs of consecutive words
            bigram = " ".join([words[i], words[i + 1]])
            if bigram == target_bigram or words[i]==target_bigram:
                start_index = max(0, i - num_words_before)
                end_index = min(len(words), i + num_words_after + 2)
                context_words.append(words[start_index:end_index])

    return [" ".join(i) for i in context_words]

In [30]:
glossary=['cabal','cultural marxism','deicide','holocough','jewish capitalist','the goyim know',
           'jewish communist','jewish lobby','new world order','rothschild', 'soros','zionist',
         'zionist occupied government','not the real jews'] 

In [33]:
new_text_unclean=[]
for text in uncleaned_texts:
    t="[CLS] "+text+" [SEP]"
    text_tokenize=tokenizer.tokenize(t)
    if len(text_tokenize)>512:
        chunks=len(text_tokenize)//512
        chunk_size=len(text.split())//(chunks+1)
        for i in range(chunks+1):
            extracted_text=" ".join(text.split(" ")[i*chunk_size:(i+1)*chunk_size])
            new_text_unclean.append(extracted_text)
    else:
        new_text_unclean.append(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors


In [81]:
expression=glossary+new_terms

In [35]:
# Generate embeddings considering complete sentence and use surrounding words for embeddings
def extract_embeddings(Posts:list[str],Expression:list[str],Range:int):
    embed_dict={i:[] for i in Expression}
    for post in Posts:
        t="[CLS] "+post+" [SEP]"
        text_tokenize=tokenizer.tokenize(t)
        if len(text_tokenize)<512:
            tensor_input_ids=torch.tensor([tokenizer.convert_tokens_to_ids(text_tokenize)])
            tensor_segment_ids= torch.tensor([[1]*len(text_tokenize)])
        
            with torch.no_grad():
                outputs=model(tensor_input_ids,tensor_segment_ids)
            token_embeddings = torch.squeeze(outputs[0])
            for term in Expression:
                if term in post:
                    sent=extract_context_words_bigram(sentence=post,target_bigram=term,num_words_before=Range,num_words_after=Range)
                    for t in sent:
                        sent_tokenize=tokenizer.tokenize(t)
                        token_index=[index for index,token in enumerate(text_tokenize) if token in sent_tokenize]
                        sent_embed=[]
                        for index in token_index:
                            sent_embed.append(token_embeddings[index])
                        if sent_embed:
                            embed=torch.mean(torch.stack(sent_embed),dim=0)
                            embed_dict[term].append(embed)
    return embed_dict

In [53]:
impembeddings=extract_embeddings(new_text_unclean,expression,Range=1)

In [42]:
no_embed_terms=[]
for i in impembeddings:
    if impembeddings[i]==[]:
        no_embed_terms.append(i)

In [43]:
no_embed_terms

['early part']

In [44]:
impterm_embeddings={}
for term in new_terms:
    if term not in no_embed_terms:
        impterm_embeddings[term]=torch.mean(torch.stack(impembeddings[term]),dim=0)

In [45]:
glossary_embeddings={}
for term in glossary:
    glossary_embeddings[term]=torch.mean(torch.stack(impembeddings[term]),dim=0)

In [46]:
column=["Term"]+glossary+["Average"]
bert_similarity_df=pd.DataFrame(columns=column)

In [47]:
for term in impterm_embeddings:
    score=0
    sim_score=[term]
    for seed_word in glossary :
        s=np.array(torch.cosine_similarity(impterm_embeddings[term].reshape(1,-1),glossary_embeddings[seed_word].reshape(1,-1)))[0]
        sim_score.append(s)
        score=score+s
    sim_score.append(score/len(glossary))
    bert_similarity_df.loc[len(bert_similarity_df)]=sim_score

In [48]:
threshold=bert_similarity_df['Average'].quantile(0.50)
bert_similarity_df['predicted']=bert_similarity_df['Average'].apply(lambda x:1 if x>threshold else 0)

In [49]:
threshold

0.8086834464754377

In [50]:
pd.DataFrame(bert_similarity_df).to_csv('/Users/dhanushkikkisetti/Documents/Research Assistant/Scripts/baseline_results1.csv')

In [56]:
predicted_labels={}
column=["Term"]+glossary+["Average"]
for r in range(1,11):
    print("Range = "+str(r))
    impembeddings=extract_embeddings(new_text_unclean,expression,Range=r)
    impterm_embeddings={}
    for term in new_terms:
        impterm_embeddings[term]=torch.mean(torch.stack(impembeddings[term]),dim=0)
    glossary_embeddings={}
    for term in glossary:
        glossary_embeddings[term]=torch.mean(torch.stack(impembeddings[term]),dim=0)
    bert_similarity_df=pd.DataFrame(columns=column)
    for term in impterm_embeddings:
        score=0
        sim_score=[term]
        for seed_word in glossary :
            s=np.array(torch.cosine_similarity(impterm_embeddings[term].reshape(1,-1),glossary_embeddings[seed_word].reshape(1,-1)))[0]
            sim_score.append(s)
            score=score+s
        sim_score.append(score/len(glossary))
        bert_similarity_df.loc[len(bert_similarity_df)]=sim_score
    threshold=bert_similarity_df['Average'].quantile(0.50)
    bert_similarity_df['predicted']=bert_similarity_df['Average'].apply(lambda x:1 if x>threshold else 0)
    predicted_labels[r]=bert_similarity_df['predicted']

Range = 1
Range = 2
Range = 3
Range = 4
Range = 5
Range = 6
Range = 7
Range = 8
Range = 9
Range = 10


In [57]:
results=pd.DataFrame.from_dict(predicted_labels)
results['avg']=results.mean(axis=1)


In [58]:
results['Predicted']=results['avg'].apply(lambda x:1 if x>0.8 else 0)

In [59]:
pd.DataFrame(results).to_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Scripts/baseline_results_1_10.csv")

In [60]:
data_file=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Scripts/baseline_results_1_10.csv")

In [61]:
print("-----------------------------------------")
print("Accuracy : ",accuracy_score(data_file['Predicted'],data_file['Actual']))
print("Precision : ",precision_score(data_file['Predicted'],data_file['Actual']))
print("Recall  : ",recall_score(data_file['Predicted'],data_file['Actual']))
print("F1 Score  : ",f1_score(data_file['Predicted'],data_file['Actual']))

-----------------------------------------
Accuracy :  0.6551724137931034
Precision :  0.7272727272727273
Recall  :  0.32
F1 Score  :  0.4444444444444444


In [63]:
for text in uncleaned_texts:
    if "great reset" in text:
        print(text)
        print("- - - - - - - - - - - - - - - - - - - - - - - - - - - -  -")

414172699 the great reset will happen next weekjfk and princess diana will come backtrump is still the presidenttrump is speaking like biden because biden will answer to trumps crimes that biden actually didthe cabal will soon be exterminatedgod bless youqtards literally live in an imaginary world
- - - - - - - - - - - - - - - - - - - - - - - - - - - -  -
articles proving putins role in the great resetglobalization putin  xi working to dugins multipolar eurasian plan global jewishcommunist slavery eastern  western jews in lockstep  kissinger  dugin jewish collapse of the west in full swing plan to annexe all western europe  destroy western civilization in preplanned ww3 deception  sinosoviet scissors strategy explained
- - - - - - - - - - - - - - - - - - - - - - - - - - - -  -
the biden clown show is useless and weak the goal of this globalist stooge is to destroy the us economy for its entry into the new world ordergreat resetliberal world order global village international world orde

### For sentence embed

In [80]:
def extract_embeddings(Posts:list[str],Expression:list[str],rAnge:int):
    terms_not_intext=[]
    embed_dict={}
    sent_dict={i:[] for i in Expression}
    for term in Expression:
        for post in Posts:
            if term in post:
                sents=extract_context_words_bigram(sentence=post,target_bigram=term,num_words_before=rAnge,num_words_after=rAnge)
                if bool(sents):
                    for sent in sents:
                        sent_dict[term].append(sent)
    for key in sent_dict:
        if bool(sent_dict[key]):
            sent_tokenize=tokenizer(sent_dict[key],return_tensors='pt',padding=True,max_length=512,truncation=True)
            with torch.no_grad():
                model_output=model(**sent_tokenize)['pooler_output']
                embed_dict[key]=model_output
        else:
            terms_not_intext.append(key)
        
    return embed_dict,terms_not_intext

In [83]:
predicted_labels={}
column=["Term"]+glossary+["Average"]
for r in range(5,15):
    print("Range = "+str(r))
    impembeddings,no_term_embed=extract_embeddings(new_text_unclean,expression,rAnge=r)
    
    impterm_embeddings={}
    for term in new_terms:
        impterm_embeddings[term]=torch.mean(impembeddings[term],dim=0)
    glossary_embeddings={}
    for term in glossary:
        glossary_embeddings[term]=torch.mean(impembeddings[term],dim=0)
    bert_similarity_df=pd.DataFrame(columns=column)
    for term in impterm_embeddings:
        score=0
        sim_score=[term]
        for seed_word in glossary :
            s=np.array(torch.cosine_similarity(impterm_embeddings[term].reshape(1,-1),glossary_embeddings[seed_word].reshape(1,-1)))[0]
            sim_score.append(s)
            score=score+s
        sim_score.append(score/len(glossary))
        bert_similarity_df.loc[len(bert_similarity_df)]=sim_score
    threshold=bert_similarity_df['Average'].quantile(0.50)
    bert_similarity_df['predicted']=bert_similarity_df['Average'].apply(lambda x:1 if x>threshold else 0)
    predicted_labels[r]=bert_similarity_df['predicted']

Range = 5
Range = 6
Range = 7
Range = 8
Range = 9
Range = 10
Range = 11
Range = 12
Range = 13
Range = 14


In [84]:
results=pd.DataFrame.from_dict(predicted_labels)
results['avg']=results.mean(axis=1)

In [85]:
results['Predicted']=results['avg'].apply(lambda x:1 if x>0.6 else 0)

In [86]:
pd.DataFrame(results).to_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Scripts/baseline_sentence_embed.csv")

In [99]:
data_file=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Scripts/baseline_results_1_10.csv")


In [100]:
print("-----------------------------------------")
print("Accuracy : ",accuracy_score(data_file['Actual'],data_file['Predicted']))
print("Precision : ",precision_score(data_file['Actual'],data_file['Predicted']))
print("Recall  : ",recall_score(data_file['Actual'],data_file['Predicted']))
print("F1 Score  : ",f1_score(data_file['Actual'],data_file['Predicted']))

-----------------------------------------
Accuracy :  0.7692307692307693
Precision :  0.3684210526315789
Recall  :  1.0
F1 Score  :  0.5384615384615384
